# CBA Policy Portal Sraper
CBA make downloading the policy difficult by
1. Having a SSO based login that makes request-http automation difficult
2. Using ASPX to dynamically load content.

Oover come this using by loggign in with the browser, then ctting and pasting the auth cookies into this notebook.

Scraper Code
1. Only searchs for links in div#wrapper
2. Has code to deal with ASPX/DynamicContent
3. Limited to commbroker domain

In [3]:
from  scraper import WebScraper
from loguru import logger
import os

class CBA(WebScraper):
    def __init__(self, domain:str, cookie:str, download_path:str, search_only:bool=False):
        super().__init__(domain, cookie, download_path, class_names = ["div#wrapper"])
        self.search_only = search_only

    def scrape(self, url:str, depth:int, starting_page:bool = True) -> None:

        logger.info(f"Scraping {depth}: {url}")
        
        if self.skip(url, depth, ["Logon.aspx"]):
            return
             
        _, extension = self.get_path(url)
        assert extension in [".html", ".aspx"], f"Invalid extension '{extension}' for {url}"

        # If aspx find the base html then extract the links and recurse without saving content
        if extension == ".aspx":
            html_urls = self.aspx_link_to_html_link(url)
            for html_url in html_urls:
                self.scrape(html_url, depth=depth, starting_page=starting_page)
            return
        
        # download unless except starting page
        html_content = self.save_url(url, starting_page)
        
        # extract all the links and recurse
        self.extract(html_content, depth, "div#wrapper")
        
        return

# Download Comm Broker

In [4]:
import shutil

directory_path=r"/home/kevinmcisaac/Projects/policy-pal-pages/policy/CBA/CommBroker/"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)  
os.makedirs(directory_path) 

#Need depth 3 to get to 
cba = CBA("https://www.commbroker.com.au", 
              cookie="cbpopup=50; s_cc=true; s_nr=1735871648417; s_sq=%5B%5BB%5D%5D; ASP.NET_SessionId=4znjrmre4pflwgcjqikk5vpk; cbid=e15hU80nELIi0HWXuqQpUEFyaPBFccqlOCX3s-c-lbGDSZecOnYqU8ys49CtD35wZz5HLWX3YhkoYxqtqCCZjX_n6HsJOOVq1loLgK6SCld7Y80NvErHo87SkJ7bp8Q3T6Bz0rtykS--OxtTB7L71Xf8OND5no9HYgGU66WdTf7MOLokeXLUlkY9lIKcVeydM0N9eTfUlUzOrA9MM1wnpXvhR7yCtRfgSJtiRfzWR9WoixaFYCgzPyX4QnMgpZsgBK9doSZ4aJivdIK_cIlbIbOiH2-No75Rh-EmE2nl3hzoDJ_OT4GpVtbsbwU6d2y-vjYWAhmcg7bSkgm3cvl0mvvZharMCKHWfrEA83Lgd2JFDWoHeHq6UJ4pl9tObn5Pex4Wjc6bLJUCClua3YGn4GB4pGTwh6s9tloWvOtuvDi0ixh94fHJhiulpw3G3uVQwjOhDmsD7OvSXkpnKolD4uGhfzeuHei4wZd3RFGnNZsF3twlBLxFpHd4C-ICc7nr97Jy72vVI790cAerq-1z0vZ3TPaQjt0JFg98OuHXBtwDFWD_PXaoEkgucvEO8tBWqF-QYwL-jiLaz3HualA9L-olA7v_ngYwPbeOpxfTJNy0dafmm6r9HJoLi-XqkVyyU-P12IbWxE9v72PjimgWRrI_ZHr03W9TlC58eAbJW-fZSP3aFWWLQYzf6v8LcVzRifCkVfN-mR6XnvWTARgZceI9uqfI12kZcPttY82OefoaPfF-Y_KK8dihY0O67Xcoi33XCqAU2iIUvJNH9icmkwFsqGQVHo9z-B6YUoIT5trXwrZE1FOID0bbP_eH5Lq4Uug_jJGK8hY-cQaGaTstLevnjYnmswRuDLRVBtkVgYSqzQYfms9Wab1j_CDwjOYJsf75dUWPp60AQkSYacEGQBE4TiFsfGCE0tHvYRFgZZOSPyuhVbM6BI49LZgxnKlkdTsnPSDkJi7agA2Dh2BEsGvbIJj_914nWheK4SIT1F71jvDa_xm1HRdDaXjoKHvLbIptEVhrMDX7_qhCcYqQg2KmXO4piE93_yxXuK7NytrALh-EWq1fKE-8JASZkCWvBk6evaRqTrxYGqTGp51KeTfcZp2RWqjiR-9IlyimEKdvUu4HNR3dQFiVSr0b2eiY91AfclXa_uJOT0JeIACJVUXbDkfoUnFKGPUlZSlOy1PCazjZYhuZER3tmhRk_gyUXtzrEg4-Ivvft80EH2Ah_LozVlquOueX7fqafV9drqDfdjPjyeev-s7bHHz3ozRwYmyjTSrK7mC57M6QM5EZwEyUbez0FNGxA0KYT1ORga3XB5IfHhM-t3lAJAnXtEvf2684BM7aWQKzeh4uVkz3hp6-z19BQcCG5LBwyvQZ5fyDKPBmoHhO86HbyFLOxtYucH8GPCinPezp73o7yVKOukibzgIMcAS3vtdIx6OkY8GjWBTD4Qa4yRPNxKVl8Y2Wi0KDnpZUtnsBLHOpSiCU6N9GDi4IqHA3qeAQIK6Dbg2qHUtQUYlzrG6BdicP6S3fqRKSwZm_XH7hFpC_wknrTCNfMxI701NDk0k6BPZYrmZaE7G9202TtRbVqPxpZ06s1oCmlVMThIz-8mlJBrZFN_jMhcfxHnXL5n8DYtR_sr-DQvbpLWFSMpMxmzBcn9gGY5LtZsGVih2lMrgBwxCaDp8RV_WMQTvB8gXIbIf7sE5TN8vmRMcciaM0mQGqeeesHMRpxrlnTfigNnRjgq3RXpRQk8k-_sUrsOzlAQGxOUsu9NDufHNldAuNe1i1-OfmCwixgEqOQ2zP_R5gtu4gsUK35An17KwaXMR-Jfv6JZoJgCukB1Umcbiyn-AnNH2x0nMVIMQ5hT_xrcsBW-jNAUOqeSN6O8GZpBlH5XbGD2XE6to06mLzO4mFtbjqCUq8DLP5Ojo02ef4ab9FG8Fvoo1tw9w8J4OV7I9CpeAQ0lKB2eMKA_6ESsE8WgOHZ2Hp76Dw4p8n33y5VZxw9t6954ve9FXujNtAIVqfioMTcM4k4uwNv9RePzFcOzaMZzz0NfyyKjLv5n1TSuRYucHZvVXUtbzkht8Drmi0mPpn1Hki7zepaudoxGibTFGAfn1Z4D-sK6_X07XaiVrjLuYgSS_szRLakF8lWSTBHqEGksCXAG5Ex4nkRO3a-fzGNj4GcLSHvCbhP_T2IA2J3mr3DteGBAdu8rLH_vaTQWIQB0jnc4UL7RI0A-6Uxd4-2FqwhfU9R_SIdML3hLWj6XeCORky-Ac4jGzm3Wut9nklu6_wJg4bpSadKDITWzE7WGDpUnRfilfz2qQG452FdMrXIQb8HUFeJLrzhe4BwrqeF4HuZamz4B-MzOTUpJhe6JqMxV46GgDjqoI1wxlJFhOR-FvSvohyQXCAgpRv4i2UJc9CuD-wb25SQhNbxBMzdswicq-UT4VWK6mU50fK6CcZ9w4p_gxMdvVdWCdT37vS7PTiKK9JElPb-meGfmCTJssCbY1aZS6wIOrbxFHIUE9pYNXeKy3mif-9zpbmW_i6jZv37Z1viAZxTp9mrwCCU2zrz2QphTnmCK5YKBxC2-zZqHl7qvaH5aLY8UHnQRn-CK9RPw2KlRm_cgz8dhZ_eUjQEhgJYMJrqR7Rjymkp83TyDjwf5HZ9hAVAj5JCQDkGhFSeEf32NO5z6KX7J_RaO4WVPmB2UOzEZfbPwzDVa_u4CCWCyDaK3vluVgmIFlRbpZraI8fBCd6kJHabF-CgRB7nSgGI8fjMm3gQH3IGKue8h9w9mRmmZdl-kFaaRAka2Y6e8KfiKPPkxs2E4J5L6UGOl-GqVXpTF7NpPRhealjsOM6iuAIJJ0-JTTKO4KegAHps-JjGizJnAdLy71JpEfy7tij8ipVEXGSak06CUNf7c5a83N3vEvz-F95xCD2wYa1SQzFNAcwcYUeYqs2xh4q03Dad1oP36b6ABvkBwUvOI9wFi-o9ZAmyh-Bk5gWC5CwwYq5BdWrpqf6TsjDyYvQSnSzFAOXyp-7VX8_wue-hYT7fClRCaa4f0riZu0BkN4i4WgpECB3-58dv4xop-Tz5bMACXShS1nNRIvVytnAfqM-Gg19mdiHqDsdH2zA_vmpZBTUWhtcRM12zR--s-bnQlRPm0YJVLvIY69if_56PXud-cA8nsao9_Tkobk_sG1g1nrCpap0pRVH7k-bo3AY2KBH3-P13yoP6nsO7ilYid32nPBSd4IdK1hVt0zFWNkaCUrv3O0QhG1yhYkvdh69UbD8IPJURPDaPsk5nIMnOZa2G4i74wY1f4BK0maUPMVApShl3OilnHI3JS4pyp9OV-sGirG15qkLA9c83gbaA5U8IILnT6l1yaYa9ITiqek5yCQldDvQhrNMgQCRVZmce9_KqWWhYvm8sekPKePKxKTeHOxd4J4gk0G-WWy9mZGGaFaGEQbIMBKVZYm4kny2g9_tbA; htsm=exp=1736119699~acl=/*~hmac=28d16e4f3f0a06aea543141f211c7b581cef0f9cf9fd17e1b214cca183ab9437; ADRUM_BTa=R:42|g:652ea9fe-f1fb-499b-83bb-71ad07c1e3c8|n:cba-prod_0e85eb6c-1d2c-4a35-a7e2-92db3c234685; SameSite=None; bm_mi=620ABCEB2AD6F65859BC67B2C6A21418~YAAQXgNiaLEIMzaUAQAA99GwOBoQllTynIcULKM884jkwkGU/lMVu0E6+bGDKmYR4jxEarZUmnEtC3VSUU1crneBXsXN0nux1QeBJgsZ8Os/qfZaaCaUb8tvYNrCvB4IZlTiyZlORwIc3p2A2/owxWuxfDbP+ZiUqRsxkqoO8HBSAZA3Yg8U42BqsjbeY7e4Nee6WvbtpH0u84SO1eiLW6tnmixDoETIBU9pMYOYq891bE8ZXvycpd7QEmjifqiGL6nNgUMF79TELIEW/ktqk2k4s6ByflevJM7tNjjJX1xsyiP71WRhmq7VJntTt949k/rb21OwBhZMoBcgTXFDWYpdGqvsVabA6is6T/AzLXf3yDI/OcKgWEuZixZGjA==~1; bm_sz=4574137452F77043785CED07ADA9B2D9~YAAQXgNiaLMIMzaUAQAA99GwOBod5Q/bNl1TLTRM7Nfbj+kKGdxe/X5PdHXhrSEpQwxyD5nrF8cISrI8lVfbnd0UjHTBhcl2P2R3vu5HeU1fp+d1SWq1sUJBZhax/i508L2KYHpIJDQCtS6Zf4PCKG+3sB/Gddup49PYGo3udrc60sYrlbRS8B6GJQf0zryUacWQJ8JZQd31RKyBnmweF8gzKeztHtZPmaiSjllwOXZ0szCiM4FiMeVM0zkvr41OjokcJ2UizRz+rKlbj27f0NkTaJjKsnwXWWUi36a0YK+Xquyyf/4efDy78s/g3lyohQNv6qSqXzjJ6LkzcWqCIyMeW41wbELwx0+8tX74aOxCzptrVPS4m7DQ4ancri/K6gD4SV4HeGvVyTIu9grVAwb0QmO/VGsZrWUPmXb0Rw==~3617336~3290680; ak_bmsc=1E37EB6A8E57C944E807F37C779550CD~000000000000000000000000000000~YAAQXgNiaC4JMzaUAQAA09WwOBrVTJPMh12s9m1CqrkuojXOOUivBUolAAiZUgBsiU24tvil52ntAp9DPCboIUKwI/UNrLriOkWI2AbXKGEHUJNnQTmio7zT4KQ/jMhhwakNdyRm6x7kSB+iX9bInao9/oTWZDgn18K0w7IuYl81O5Yge51ZvHiKH382030wT7ciHYUvAF3T45ZpO6lTod8kMmXH6/TTUfQ9V+0NTNz8KUrPmZg3eNUkQOzGRx0ABkp0raPEdXIGCvf5G9KzCcGKYLA3XIwr/DB+aadaSFepBddg5VxV05mMRKyumKul2N3mAkQBocHFBlOquV8ZlsngkXG35/73+C3ONLS9J5ILL32l/cEXeZmjdINoJoM6HiwPigQ5SG38u2M3tEOhGD6Z4v0nwCTpu2Wm5xm65K2vrNv/5bvelXbeR+qbZE0djqfGW0n6RJZ+ScvzA5MiedaAy7SFdxeLa6j1YYvGCEiW59M9Q4WeuzdxQ4cnYPhFUcI+t0HnsCah8zl33r1tloq9pFMc1ds=; bm_sv=FD5A116729532935542ABC94B4C5F512~YAAQRQNiaCj/+DeUAQAAm9mwOBrhmQr1bdXVKWb4sOdvMQTy11HZJrU4kApWB8PXH4IntyFM2dBCwyGjWcvuGHOfTI4+AZ25Gwm1sC9J6lFm4n8n2G11et4muUfInxOTOG5hFDgoh7uQtqphI5+gMP2XoxS3Me/jF1gjeqIuqL7IcpQQP0j4gGVCFtlt6OsYl/r4yRrdW/HjaGJmjLWbsYL3ULnTamefqpoMB5t6KNT9+UOR9dHVb/opvIeotboG+tiJQHUMGg==~1; _abck=CDA8A3A4C475B653B887DAA636D183BF~-1~YAAQRQNiaLT/+DeUAQAA6t6wOA1pn8i4f13Q7cuTK6kVcD2y/tkJ9YxVpho6j6Gh2wIe/jnPgTeuO+OXT9g8u9rEXJU9QmtuYxPodZyeEjHkoqtsyFNaoC+7PTHcFlN2Nsjic+ojzsZuXVF4+4B3ey2Y0a/4zzfJWhk1NSEZ0ySeQ75zLBZRWRWE/csWg8lvmFvsOjOvoJ0Vt4K4+kOPsjD5hL3fiy1AlOOWZATtzys0TMvOQnz0RsYgoy3L9QKio1wnQ7qEY95S9vSIVWrSoerdL1Qrib0+ZzxdYaBf+qHaVyMEOsq69ZzgakAnPPvGT6M33eNJnbtPTfM9vvImgONt96JpSu6CNDj4oqswLsbe4Wjzudry5tcFmdmnqINECCY/WDRRu39SCO50YxG3HqlpET2epsp3QzWBtXb1KLBFXNnY45bZBfq8u5/U4lv3B2n2/2H7zbEdMUFKh1tdF8E3BKoQ2oKLr5/WPRTcoUp8fg458YzU6m9K1HOTDv1xQPwBQ+efl79Sek4JdLU=~-1~-1~-1", 
              download_path= directory_path, 
              search_only=False)
cba.scrape("https://www.commbroker.com.au/Net/Documentum/credit-policy/default.aspx", depth=16)
cba.scrape("https://www.commbroker.com.au/Net/Documentum/processes/default.aspx", depth=16)

print(len(cba.download))

2025-01-06 07:40:31.060 | INFO     | __main__:scrape:12 - Scraping 16: https://www.commbroker.com.au/Net/Documentum/credit-policy/default.aspx
2025-01-06 07:40:31.817 | INFO     | __main__:scrape:12 - Scraping 16: https://www.commbroker.com.au/content/commbroker/en/private/credit-policy/default.html
2025-01-06 07:40:31.969 | INFO     | __main__:scrape:12 - Scraping 15: https://ecomms.cba.com.au/cv/4e28c09365e1c179bd89ab131200ec829530c314
2025-01-06 07:40:31.971 | INFO     | scraper:skip:118 - Skipping domain https://ecomms.cba.com.au/cv/4e28c09365e1c179bd89ab131200ec829530c314
2025-01-06 07:40:31.975 | INFO     | __main__:scrape:12 - Scraping 15: https://www.commbroker.com.au/Net/Documentum/credit-policy/acceptable-excluded-purposes.aspx
2025-01-06 07:40:32.208 | INFO     | __main__:scrape:12 - Scraping 15: https://www.commbroker.com.au/content/commbroker/en/private/credit-policy/acceptable-excluded-purposes.html
2025-01-06 07:40:32.452 | INFO     | scraper:save_url:64 - Downloading ht

217
